In [1]:
%load_ext autoreload
%autoreload 2

from utils import *
import pyclesperanto as cle
import numpy as np
import pandas as pd
cle.select_device("RTX")

(OpenCL) NVIDIA GeForce RTX 3060 Laptop GPU (OpenCL 3.0 CUDA)
	Vendor:                      NVIDIA Corporation
	Driver Version:              516.95
	Device Type:                 GPU
	Compute Units:               30
	Global Memory Size:          6143 MB
	Maximum Object Size:         1535 MB
	Max Clock Frequency:         1425 MHz
	Image Support:               Yes

In [2]:
path = "../../04_uCT/AD51/"
scan = load_scan(path, logging=True)
# scan.reslice()

Loading images from:  ../../04_uCT/AD51/Slices/
Loaded stack with shape (1780, 322, 2267) and a size of 2.60 GB in 4.89 s.


In [3]:
segment_scan(scan[145:150])

Smoothed image.
Finished otsu masks.
Finished refined particle mask.
Created full segmentation.


array([[[2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.],
        ...,
        [2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.]],

       [[2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.],
        ...,
        [2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.]],

       [[2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.],
        ...,
        [2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.]],

       [[2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.],
        ...,
        [2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.],
        [2., 2., 2., ..., 2., 2., 2.]],

    

In [4]:
# show_in_napari(scan[145:150], )